# SQL Practice- Chinook Database

In [1]:
import sqlite3
import pandas as pd

In [2]:
#establish connection to the database
conn = sqlite3.connect('chinook.db')
c = conn.cursor()

In [3]:
sqlite_query = """
SELECT name FROM sqlite_master WHERE type = 'table';
"""
pd.read_sql(sqlite_query, conn)

,name
0,albums
1,sqlite_sequence
2,artists
3,customers
4,employees
5,genres
6,invoices
7,invoice_items
8,media_types
9,playlists


12 different datasets given in Chinook Database

In [4]:
table_names = pd.read_sql(sqlite_query, conn)['name']

In [35]:
#gives the top 5 rows of each table in the database
for i in table_names:
    sqlite_query = """
    SELECT * from {} LIMIT 5;""".format(i)
    print("\n\n\n" + i + "\n")
    print(pd.read_sql(sqlite_query, conn))




albums

   AlbumId                                  Title  ArtistId
0        1  For Those About To Rock We Salute You         1
1        2                      Balls to the Wall         2
2        3                      Restless and Wild         2
3        4                      Let There Be Rock         1
4        5                               Big Ones         3



sqlite_sequence

          name   seq
0       genres    25
1  media_types     5
2      artists   275
3       albums   347
4       tracks  3503



artists

   ArtistId               Name
0         1              AC/DC
1         2             Accept
2         3          Aerosmith
3         4  Alanis Morissette
4         5    Alice In Chains



customers

   CustomerId  FirstName     LastName  \
0           1       Luís    Gonçalves   
1           2     Leonie       Köhler   
2           3   François     Tremblay   
3           4      Bjørn       Hansen   
4           5  František  Wichterlová   

                        

In [6]:
sqlite_query = """
SELECT COUNT(DISTINCT(AlbumID)) as '#Distinct Albums', COUNT(DISTINCT(ArtistId)) as '#Disinct Artists' FROM albums;
"""
pd.read_sql(sqlite_query, conn)

,#Distinct Albums,#Disinct Artists
0,347,204


204 artists produce 347 albums

In [34]:
sqlite_query = """
SELECT * FROM sqlite_sequence;"""
pd.read_sql(sqlite_query, conn)

,name,seq
0,genres,25
1,media_types,5
2,artists,275
3,albums,347
4,tracks,3503
5,employees,8
6,customers,59
7,invoices,412
8,invoice_items,2240
9,playlists,18


From inspection this appears to give the number of the different genres, media_types, artists, ... in the database.

In [33]:
sqlite_query = """
SELECT Name as "artist_name", COUNT(*) as "number_of_albums"
FROM albums 
INNER JOIN artists
ON artists.ArtistId = albums.ArtistId
GROUP BY albums.ArtistID
ORDER BY number_of_albums DESC
LIMIT 10;
"""
pd.read_sql(sqlite_query, conn)

,artist_name,number_of_albums
0,Iron Maiden,21
1,Led Zeppelin,14
2,Deep Purple,11
3,Metallica,10
4,U2,10
5,Ozzy Osbourne,6
6,Pearl Jam,5
7,Various Artists,4
8,Faith No More,4
9,Foo Fighters,4


## Toy exercises

### What is the most popular genre in the US and Canada?

In [32]:
#seeing if any of the tracks in the invoice_items were ordered in a quantity of more than one
sql_query = """ 
SELECT quantity, COUNT(*)
FROM invoice_items 
GROUP BY quantity;
"""
pd.read_sql(sql_query, conn)


,Quantity,COUNT(*)
0,1,2240


In [31]:
#how many orders are from people in the USA or Canada
sql_query = """
SELECT COUNT(*) as "number_of_orders"
FROM invoices
INNER JOIN customers
ON customers.CustomerId = invoices.CustomerId
WHERE customers.country = "USA" or customers.country = "Canada";
"""
pd.read_sql(sql_query, conn)

,number_of_orders
0,147


Note these orders can consist of multiple tracks

In [30]:
sql_query = """ 
SELECT genres.name, count(*) as "genre_frequency"
FROM invoice_items
INNER JOIN tracks
ON tracks.TrackId = invoice_items.TrackId
INNER JOIN genres 
ON genres.GenreId = tracks.GenreId
INNER JOIN invoices
ON invoices.InvoiceId = invoice_items.InvoiceId
INNER JOIN customers 
ON customers.customerId = invoices.customerId
WHERE customers.country =  "USA" OR customers.country = "Canada"
GROUP BY genres.name
ORDER BY genre_frequency DESC
LIMIT 10;
"""
print("USA or CANADA")
pd.read_sql(sql_query, conn)

USA or CANADA


,Name,genre_frequency
0,Rock,264
1,Latin,151
2,Metal,104
3,Alternative & Punk,86
4,Jazz,35
5,Blues,19
6,R&B/Soul,17
7,TV Shows,15
8,Bossa Nova,14
9,Classical,13


Rock appears to be the most popular genre in the USA and Canada within the dataset.

In [29]:
#overall frequency
sql_query = """ 
SELECT genres.name, count(*) as "genre_frequency"
FROM invoice_items
INNER JOIN tracks
ON tracks.TrackId = invoice_items.TrackId
INNER JOIN genres 
ON genres.GenreId = tracks.GenreId
INNER JOIN invoices
ON invoices.InvoiceId = invoice_items.InvoiceId
INNER JOIN customers 
ON customers.customerId = invoices.customerId
GROUP BY genres.name
ORDER BY genre_frequency DESC
LIMIT 10;
"""

pd.read_sql(sql_query, conn)

,Name,genre_frequency
0,Rock,835
1,Latin,386
2,Metal,264
3,Alternative & Punk,244
4,Jazz,80
5,Blues,61
6,TV Shows,47
7,Classical,41
8,R&B/Soul,41
9,Reggae,30


### Who is the top selling artist of 2012?

In [28]:
sql_query = """
SELECT artists.name, count(*) as "freq", strftime('%Y', InvoiceDate) as "year" FROM invoices
INNER JOIN invoice_items
ON invoice_items.invoiceid = invoices.invoiceid
INNER JOIN tracks
ON invoice_items.trackid = tracks.trackid
INNER JOIN albums
ON albums.albumid = tracks.albumid
INNER JOIN artists
ON artists.artistid = albums.artistid
WHERE year = '2012'
GROUP BY (artists.name)
ORDER BY freq DESC
LIMIT 1;
"""
pd.read_sql(sql_query, conn)

,Name,freq,year
0,Iron Maiden,34,2012


### Which customer spent the most overall in the data?

In [27]:
sql_query = """
SELECT invoices.customerid, customers.firstname || ' ' || customers.lastname as "customer_name", SUM(invoice_items.unitprice * invoice_items.quantity) as "amount_spent" 
FROM invoice_items 
INNER JOIN invoices
ON invoice_items.invoiceid = invoices.invoiceid
INNER JOIN customers
ON customers.customerid = invoices.customerid
GROUP BY invoices.customerid
ORDER BY amount_spent DESC
LIMIT 1;
"""
pd.read_sql(sql_query, conn)

,CustomerId,customer_name,amount_spent
0,6,Helena Holý,49.62


### How many customers spent over £40?

In [26]:
sql_query = """
SELECT COUNT(*) FROM
(SELECT invoices.customerid, SUM(invoice_items.unitprice * invoice_items.quantity) as "amount_spent" 
FROM invoice_items 
INNER JOIN invoices
ON invoice_items.invoiceid = invoices.invoiceid
INNER JOIN customers
ON customers.customerid = invoices.customerid
GROUP BY invoices.customerid)
WHERE amount_spent >= 40;
"""
pd.read_sql(sql_query, conn)

,COUNT(*)
0,14


### What is the percentage change between the amount spent on sales of tracks between 2012 and 2011?

In [25]:
sql_query = """
SELECT strftime('%Y', invoicedate) as "year", SUM(total) FROM invoices
GROUP BY year;"""
pd.read_sql(sql_query, conn)

,year,SUM(total)
0,2009,449.46
1,2010,481.45
2,2011,469.58
3,2012,477.53
4,2013,450.58


In [17]:
percentage_increase = (477.53/469.58 - 1 )*100.0
print('There is a percentage increase between 2011 and 2012 of {}%.'.format(round(percentage_increase,2)))

There is a percentage increase between 2011 and 2012 of 1.69%.


### How many customers have gmail accounts?

In [49]:
sql_query = """
SELECT COUNT(email) as "#gmail_acounts" FROM customers 
WHERE email LIKE '%gmail%';
"""
pd.read_sql(sql_query, conn)

,#gmail_acounts
0,8


### Show a frequency table of the different roles of employees as a percentage of the overall number of employees

In [94]:
sql_query = """ 
SELECT COUNT(*) 
FROM employees
"""
pd.read_sql(sql_query, conn)

,COUNT(*)
0,8


In [99]:
sql_query = """
SELECT title, (COUNT(*)/8.0 *100) as "percentage"
FROM employees
GROUP BY title;
"""
pd.read_sql(sql_query, conn)

,Title,percentage
0,General Manager,12.5
1,IT Manager,12.5
2,IT Staff,25.0
3,Sales Manager,12.5
4,Sales Support Agent,37.5


In [100]:
conn.close()